# 1. Importing modules and functions

In [108]:
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from molvs import standardize_smiles
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.model_selection import permutation_test_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_predict
from sklearn import metrics
from sklearn.metrics import pairwise_distances
import joblib
import pickle
from numpy import savetxt
from padelpy import from_sdf
import shap
from tqdm.notebook import tqdm

# 2.Data entry and curation work set

In [109]:
uploaded_file_ws="datasets/HDAC6_work.sdf"
supplier_ws = Chem.ForwardSDMolSupplier(uploaded_file_ws,sanitize=False)
failed_mols_ws = []
all_mols_ws =[]
wrong_structure_ws=[]
wrong_smiles_ws=[]
y_tr = []
y_bad_index=[]

for i, m in enumerate(supplier_ws):
    structure = Chem.Mol(m)
    all_mols_ws.append(structure)
    y_tr.append(m.GetProp("pchembl_value_mean"))
    try:
        Chem.SanitizeMol(structure)
    except:
        failed_mols_ws.append(m)
        wrong_smiles_ws.append(Chem.MolToSmiles(m))
        wrong_structure_ws.append(str(i+1))
        y_bad_index.append(i)
print('Original data: ', len(all_mols_ws), 'molecules')
print('Failed data: ', len(failed_mols_ws), 'molecules')
number_ws =[]
for i in range(len(failed_mols_ws)):
        number_ws.append(str(i+1))
bad_molecules_ws = pd.DataFrame({'No. failed molecule in original set': wrong_structure_ws, 'SMILES of wrong structure: ': wrong_smiles_ws, 'No.': number_ws}, index=None)
bad_molecules_ws = bad_molecules_ws.set_index('No.')
bad_molecules_ws

Original data:  3083 molecules
Failed data:  0 molecules


,No. failed molecule in original set,SMILES of wrong structure:
No.,,


deleting activity values for substances with incorrect structure

In [110]:
y_tr[:] = [x for i,x in enumerate(y_tr) if i not in y_bad_index]

In [111]:
len(y_tr)

3083

# 3.Standardization SDF file for work set

In [112]:
all_mols_ws[:] = [x for i,x in enumerate(all_mols_ws) if i not in y_bad_index] 
records = []
for i in range(len(all_mols_ws)):
    record = Chem.MolToSmiles(all_mols_ws[i])
    records.append(record)

moldf_ws = []
for i,record in enumerate(records):
    standard_record = standardize_smiles(record)
    m = Chem.MolFromSmiles(standard_record)
    moldf_ws.append(m)
    
print('Kept data: ', len(moldf_ws), 'molecules')

Kept data:  3083 molecules


In [113]:
records_ws = []
for i in range(len(moldf_ws)):
    record = Chem.MolToSmiles(moldf_ws[i])
    records_ws.append(record)

In [114]:
df = pd.DataFrame(records_ws, columns=["Smiles"])
df.to_csv('datasets/molecule_ws.smi', sep=',', index=False, header=False)

# 4.Data entry and curation test set

In [115]:
uploaded_file_ts="datasets/HDAC6_test.sdf"
supplier_ts = Chem.ForwardSDMolSupplier(uploaded_file_ts,sanitize=False)
failed_mols_ts = []
all_mols_ts =[]
wrong_structure_ts=[]
wrong_smiles_ts=[]
y_ts = []
y_bad_index=[]
for i, m in enumerate(supplier_ts):
    structure = Chem.Mol(m)
    all_mols_ts.append(structure)
    y_ts.append(m.GetProp("pchembl_value_mean"))
    try:
        Chem.SanitizeMol(structure)
    except:
        failed_mols_ts.append(m)
        wrong_smiles_ts.append(Chem.MolToSmiles(m))
        wrong_structure_ts.append(str(i+1))
        y_bad_index.append(i)
print('Original data: ', len(all_mols_ts), 'molecules')
print('Failed data: ', len(failed_mols_ts), 'molecules')
number_ts =[]
for i in range(len(failed_mols_ts)):
        number_ts.append(str(i+1))
bad_molecules_ts = pd.DataFrame({'No. failed molecule in original set': wrong_structure_ts, 'SMILES of wrong structure: ': wrong_smiles_ts, 'No.': number_ts}, index=None)
bad_molecules_ts = bad_molecules_ts.set_index('No.')
bad_molecules_ts

Original data:  771 molecules
Failed data:  0 molecules


,No. failed molecule in original set,SMILES of wrong structure:
No.,,


deleting activity values for substances with incorrect structure

In [116]:
y_ts[:] = [x for i,x in enumerate(y_ts) if i not in y_bad_index]

In [117]:
len(y_ts)

771

# 5.Standardization SDF file for test set

In [118]:
all_mols_ts[:] = [x for i,x in enumerate(all_mols_ts) if i not in y_bad_index] 
records = []
for i in range(len(all_mols_ts)):
    record = Chem.MolToSmiles(all_mols_ts[i])
    records.append(record)

moldf_ts = []
for i,record in enumerate(records):
    standard_record = standardize_smiles(record)
    m = Chem.MolFromSmiles(standard_record)
    moldf_ts.append(m)
    
print('Kept data: ', len(moldf_ts), 'molecules')

Kept data:  771 molecules


In [119]:
records_ts = []
for i in range(len(moldf_ts)):
    record = Chem.MolToSmiles(moldf_ts[i])
    records_ts.append(record)

In [14]:
df_ts = pd.DataFrame(records_ts, columns=["Smiles"])
df_ts.to_csv('datasets/molecule_ts.smi', sep=',', index=False, header=False)

# 6.Descriptor calculation for work set

## AtomPairs2D FPs

In [163]:
from padelpy import padeldescriptor

fingerprint = 'AtomPairs2D'

fingerprint_output_file = ''.join([fingerprint,'.csv']) #Substructure.csv
fingerprint_descriptortypes = fp[fingerprint]

padeldescriptor(mol_dir='datasets/molecule_ws.smi', 
                d_file=fingerprint_output_file, #'Substructure.csv'
                #descriptortypes='SubstructureFingerprint.xml', 
                descriptortypes= fingerprint_descriptortypes,
                detectaromaticity=True,
                standardizenitro=True,
                standardizetautomers=True,
                threads=2,
                removesalt=True,
                log=True,
                fingerprints=True)

In [120]:
descriptors_AtomPairs2D = pd.read_csv('AtomPairs2D.csv')

In [121]:
descriptors_AtomPairs2D

,Name,AD2D1,AD2D2,AD2D3,AD2D4,AD2D5,AD2D6,AD2D7,AD2D8,AD2D9,...,AD2D771,AD2D772,AD2D773,AD2D774,AD2D775,AD2D776,AD2D777,AD2D778,AD2D779,AD2D780
0,AUTOGEN_molecule_ws_1,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,AUTOGEN_molecule_ws_2,1,1,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,AUTOGEN_molecule_ws_3,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,AUTOGEN_molecule_ws_4,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,AUTOGEN_molecule_ws_5,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3078,AUTOGEN_molecule_ws_3079,1,1,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3079,AUTOGEN_molecule_ws_3080,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3080,AUTOGEN_molecule_ws_3081,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3081,AUTOGEN_molecule_ws_3082,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [122]:
x_tr = descriptors_AtomPairs2D.drop('Name', axis=1)

In [123]:
x_tr = np.array(x_tr, dtype=np.float32)
y_tr = np.array(y_tr, dtype=np.float32)

In [124]:
savetxt('Models/Padels/x_tr_AtomPairs2D.csv', y_tr, delimiter=',')

In [55]:
x_tr.shape

(3083, 780)

# 7.Descriptor calculation for test set

In [179]:
from padelpy import padeldescriptor

fingerprint = 'AtomPairs2D'

fingerprint_output_file = ''.join([fingerprint,'_ts.csv']) #AtomPairs2D.csv
fingerprint_descriptortypes = fp[fingerprint]

padeldescriptor(mol_dir='datasets/molecule_ts.smi', 
                d_file=fingerprint_output_file, #'AtomPairs2D.csv'
                #descriptortypes='AtomPairs2D.xml', 
                descriptortypes= fingerprint_descriptortypes,
                detectaromaticity=True,
                standardizenitro=True,
                standardizetautomers=True,
                threads=2,
                removesalt=True,
                log=False,
                fingerprints=True)

In [125]:
descriptors_AtomPairs2D_ts = pd.read_csv('AtomPairs2D_ts.csv')

In [126]:
x_ts = descriptors_AtomPairs2D_ts.drop('Name', axis=1)

In [127]:
x_ts.shape

(771, 780)

# load the models from disk

In [128]:
best_svm = pickle.load(open('Models/Padels/HDAC6_SVM_AtomPairs2D.pkl', 'rb'))

In [129]:
best_gbr = pickle.load(open('Models/Padels/HDAC6_GBR_AtomPairs2D.pkl', 'rb'))

In [130]:
best_MLPR = pickle.load(open('Models/Padels/HDAC6_MLPR_AtomPairs2D.pkl', 'rb'))

# Prediction for CV

In [131]:
y_pred_ws_GBR = best_gbr.predict(x_tr)

In [132]:
y_pred_ws_svm = best_svm.predict(x_tr)

In [133]:
y_pred_con=(y_pred_ws_GBR+y_pred_ws_svm)/2

In [134]:
R2_WS = round(r2_score(y_tr, y_pred_con), 2)
R2_WS

0.8

In [135]:
RMSE_WS=round(np.sqrt(mean_absolute_error(y_tr, y_pred_con)), 2)
RMSE_WS

0.59

In [136]:
seed = 42
cv=KFold(n_splits=5, random_state=seed, shuffle=True)

In [137]:
y_pred_CV_svm = cross_val_predict(best_svm, x_tr, y_tr, cv=cv)

In [138]:
y_pred_CV_gbr = cross_val_predict(best_gbr, x_tr, y_tr, cv=cv)

In [139]:
y_pred_CV_MLPR = cross_val_predict(best_MLPR, x_tr, y_tr, cv=cv)

# For all models

In [140]:
y_pred_con=(y_pred_CV_svm+y_pred_CV_gbr+y_pred_CV_MLPR)/3

In [141]:
Q2_CV = round(r2_score(y_tr, y_pred_con), 2)
Q2_CV

0.53

In [142]:
RMSE_CV=round(np.sqrt(mean_absolute_error(y_tr, y_pred_con)),2)
RMSE_CV

0.76

# two models: svm+gbr

In [143]:
y_pred_con_without_MLPR=(y_pred_CV_svm+y_pred_CV_gbr)/2

In [144]:
Q2_CV = round(r2_score(y_tr, y_pred_con_without_MLPR), 2)
Q2_CV

0.54

In [145]:
RMSE_CV=round(np.sqrt(mean_absolute_error(y_tr, y_pred_con_without_MLPR)),2)
RMSE_CV

0.75

# Prediction for test set's molecules

In [146]:
x_ts = np.array(x_ts, dtype=np.float32)
y_ts = np.array(y_ts, dtype=np.float32)

In [147]:
y_pred_svm = best_svm.predict(x_ts)

In [148]:
y_pred_gbr = best_gbr.predict(x_ts)

In [149]:
y_pred_MLPR = best_MLPR.predict(x_ts)

In [150]:
y_pred_con=(y_pred_svm+y_pred_gbr)/2

In [151]:
Q2_TS = round(r2_score(y_ts, y_pred_con), 2)
Q2_TS

0.58

In [152]:
RMSE_TS=round(np.sqrt(mean_absolute_error(y_ts, y_pred_con)), 2)
RMSE_TS

0.74

# Estimating applicability domain. Method - Euclidian distances, K=1

In [153]:
neighbors_k= pairwise_distances(x_tr, n_jobs=-1)
neighbors_k.sort(0)

In [154]:
df_tr=pd.DataFrame(neighbors_k)
df_tr

,0,1,2,3,4,5,6,7,8,9,...,3073,3074,3075,3076,3077,3078,3079,3080,3081,3082
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2.000000,4.000000,2.449490,0.000000,3.316625,0.000000,3.000000,1.732051,1.414214,3.872983,...,1.414214,2.645751,1.414214,1.000000,0.000000,0.000000,0.000000,2.449490,1.000000,1.414214
2,2.828427,4.123106,2.449490,0.000000,3.872983,3.000000,3.162278,1.732051,2.645751,4.358899,...,1.414214,2.645751,1.732051,1.000000,0.000000,2.000000,1.000000,2.645751,1.732051,1.414214
3,3.000000,4.123106,2.645751,0.000000,4.000000,3.000000,3.162278,1.732051,2.828427,4.690416,...,1.732051,3.000000,2.000000,1.000000,0.000000,2.645751,1.000000,2.645751,1.732051,1.414214
4,3.000000,4.123106,2.645751,0.000000,4.000000,3.000000,3.162278,2.000000,2.828427,4.690416,...,2.000000,3.872983,2.000000,1.000000,0.000000,3.464102,1.414214,2.828427,1.732051,1.414214
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3078,9.539392,9.486833,9.380832,9.643651,10.583005,9.486833,9.643651,9.433981,8.944272,10.198039,...,9.273619,8.602325,9.110434,9.327379,9.273619,8.717798,9.380832,8.831760,9.433981,9.327379
3079,9.591663,9.539392,9.433981,9.695360,10.630146,9.486833,9.695360,9.433981,9.110434,10.198039,...,9.273619,8.602325,9.110434,9.327379,9.273619,8.717798,9.380832,8.888194,9.433981,9.327379
3080,9.797959,9.539392,9.433981,9.746795,10.677078,9.539392,9.746795,9.486833,9.110434,10.198039,...,9.273619,8.602325,9.327379,9.380832,9.327379,8.717798,9.433981,9.000000,9.486833,9.380832
3081,9.797959,9.591663,9.643651,9.949874,10.862781,9.746795,9.746795,9.695360,9.273619,10.246951,...,9.486833,8.660254,9.433981,9.591663,9.539392,8.717798,9.643651,9.055386,9.643651,9.591663


In [155]:
similarity= neighbors_k

In [156]:
Dmean=np.mean(similarity[1,:])

In [157]:
round(Dmean, 2)

1.14

In [158]:
std=np.std(similarity[1,:])

In [159]:
round(std, 2)

1.27

In [160]:
model_AD_limit=Dmean+std*0.5
print(np.round(model_AD_limit, 2))

1.77


In [161]:
neighbors_k_ts= pairwise_distances(x_tr,Y=x_ts, n_jobs=-1)
neighbors_k_ts.sort(0)

In [162]:
x_ts_AD=pd.DataFrame(neighbors_k_ts)
x_ts_AD

,0,1,2,3,4,5,6,7,8,9,...,761,762,763,764,765,766,767,768,769,770
0,2.828427,2.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.449490,2.645751,1.000000,0.000000,0.000000,3.316625,0.000000,3.605551,1.732051,0.000000
1,3.162278,2.000000,0.000000,2.449490,1.000000,1.414214,1.414214,0.000000,0.000000,0.000000,...,3.162278,2.645751,1.000000,0.000000,0.000000,3.464102,2.645751,3.741657,1.732051,0.000000
2,3.162278,2.000000,0.000000,2.449490,1.414214,3.000000,1.414214,0.000000,0.000000,0.000000,...,3.162278,2.645751,1.732051,1.414214,0.000000,3.464102,2.645751,3.872983,2.000000,1.000000
3,3.316625,2.000000,1.000000,2.645751,2.449490,3.000000,1.414214,0.000000,3.464102,1.414214,...,3.316625,2.645751,1.732051,1.414214,0.000000,3.464102,3.000000,4.000000,2.000000,1.000000
4,3.316625,2.000000,1.414214,2.645751,2.449490,3.162278,1.414214,0.000000,3.872983,1.414214,...,3.464102,3.000000,2.000000,1.414214,0.000000,3.464102,3.872983,4.000000,2.000000,1.414214
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3078,10.099504,9.433981,9.380832,9.380832,9.273619,9.055386,9.219544,9.327379,9.539392,9.486833,...,8.888194,8.944272,9.165152,9.165152,9.380832,9.380832,8.602325,9.273619,9.165152,9.380832
3079,10.148891,9.486833,9.380832,9.380832,9.327379,9.165152,9.219544,9.380832,9.591663,9.486833,...,9.110434,8.944272,9.219544,9.165152,9.380832,9.433981,8.602325,9.273619,9.165152,9.380832
3080,10.246951,9.486833,9.433981,9.433981,9.380832,9.219544,9.273619,9.380832,9.591663,9.539392,...,9.110434,8.944272,9.219544,9.219544,9.433981,9.486833,8.602325,9.273619,9.165152,9.433981
3081,10.344080,9.695360,9.643651,9.486833,9.433981,9.219544,9.486833,9.433981,9.643651,9.746795,...,9.165152,9.000000,9.433981,9.273619,9.643651,9.539392,8.660254,9.327379,9.380832,9.643651


In [163]:
similarity_ts= neighbors_k_ts
cpd_AD=similarity_ts[0,:]
cpd_value = np.round(cpd_AD, 3)
print(cpd_value)

[2.828 2.    0.    2.    0.    0.    0.    0.    0.    0.    0.    0.
 1.732 1.732 1.    1.732 0.    0.    2.646 0.    2.449 1.    1.    0.
 3.    0.    2.646 3.162 0.    1.732 0.    1.732 1.414 1.732 3.606 1.
 0.    1.732 0.    1.732 1.    0.    0.    2.    1.    1.    2.236 0.
 0.    1.    1.732 0.    3.606 1.    0.    0.    1.414 0.    1.414 0.
 0.    1.    1.    2.236 1.414 2.449 2.449 0.    2.    1.732 5.    0.
 0.    0.    0.    1.    0.    1.    1.414 0.    1.    0.    3.162 0.
 2.828 2.236 1.414 0.    2.236 0.    2.    3.162 0.    0.    0.    2.
 2.    3.317 1.414 2.449 1.    1.414 0.    2.236 2.236 2.449 0.    2.449
 1.414 2.    1.414 3.    0.    2.646 0.    0.    0.    0.    1.    0.
 0.    0.    1.414 3.606 3.317 0.    0.    0.    0.    1.414 2.828 0.
 1.414 1.    2.449 2.646 4.69  0.    2.    0.    1.    1.    2.449 0.
 3.162 0.    2.236 0.    2.    1.    0.    0.    3.464 1.732 1.732 2.828
 2.449 2.    2.236 0.    0.    2.828 2.646 4.243 2.236 0.    3.317 1.
 0.    0.    1

In [164]:
cpd_AD = np.where(cpd_value <= model_AD_limit, True, False)
print(cpd_AD)

[False False  True False  True  True  True  True  True  True  True  True
  True  True  True  True  True  True False  True False  True  True  True
 False  True False False  True  True  True  True  True  True False  True
  True  True  True  True  True  True  True False  True  True False  True
  True  True  True  True False  True  True  True  True  True  True  True
  True  True  True False  True False False  True False  True False  True
  True  True  True  True  True  True  True  True  True  True False  True
 False False  True  True False  True False False  True  True  True False
 False False  True False  True  True  True False False False  True False
  True False  True False  True False  True  True  True  True  True  True
  True  True  True False False  True  True  True  True  True False  True
  True  True False False False  True False  True  True  True False  True
 False  True False  True False  True  True  True False  True  True False
 False False False  True  True False False False Fa

In [165]:
print("Coverage = ", sum(cpd_AD) / len(cpd_AD))

Coverage =  0.7600518806744487


In [166]:
print("Indices of substances included in AD = ", np.where(cpd_AD != 0)[0])

Indices of substances included in AD =  [  2   4   5   6   7   8   9  10  11  12  13  14  15  16  17  19  21  22
  23  25  28  29  30  31  32  33  35  36  37  38  39  40  41  42  44  45
  47  48  49  50  51  53  54  55  56  57  58  59  60  61  62  64  67  69
  71  72  73  74  75  76  77  78  79  80  81  83  86  87  89  92  93  94
  98 100 101 102 106 108 110 112 114 115 116 117 118 119 120 121 122 125
 126 127 128 129 131 132 133 137 139 140 141 143 145 147 149 150 151 153
 154 159 160 165 167 168 169 170 172 174 176 177 178 180 181 182 183 184
 185 186 188 189 191 193 195 196 197 198 199 201 202 203 206 207 208 210
 212 213 214 215 217 218 219 220 221 222 223 224 226 227 228 229 230 231
 232 233 234 235 236 237 238 239 240 241 242 244 245 246 247 248 249 250
 251 252 253 254 255 256 257 259 261 262 264 265 266 267 270 272 273 275
 276 277 278 282 283 284 286 287 288 289 290 291 292 293 294 295 297 298
 299 300 301 302 303 304 305 306 308 309 310 313 314 316 317 318 319 320
 321 325 32

In [167]:
out_Ad=list(np.where(cpd_AD == 0)[0])

# Prediction only for molecules included in  AD

In [168]:
y_pred_con_ad=list(y_pred_con)

In [169]:
y_pred_con_ad[:] = [x for i,x in enumerate(y_pred_con_ad) if i not in out_Ad]

In [170]:
len(y_pred_con_ad)

586

In [171]:
y_ts_ad=list(y_ts)

In [172]:
y_ts_ad[:] = [x for i,x in enumerate(y_ts_ad) if i not in out_Ad]

In [173]:
len(y_ts_ad)

586

In [174]:
Q2_TS = round(r2_score(y_ts_ad, y_pred_con_ad), 2)
Q2_TS

0.62

In [175]:
RMSE_TS=round(np.sqrt(mean_absolute_error(y_ts_ad, y_pred_con_ad)), 2)
RMSE_TS

0.71